In [1]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading
import subprocess
import psutil

import warnings
warnings.filterwarnings("ignore", message="Dependency not available for matrix_profile")

# Initialize the lock globally
mt5_lock = threading.Lock()

logging.basicConfig(filename='trading_EURUSD_GBPUSD_USDCAD_AUDUSD_USDCHF_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Kill any existing MetaTrader 5 instances
def kill_mt5_instances():
    for process in psutil.process_iter():
        try:
            if "terminal64.exe" in process.name().lower():
                process.kill()
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass  # Ignore these exceptions

# Initialize connection to MetaTrader 5 with proper locking and multiple instances handling
def init_mt5_connection(login, password, server, terminal_path):
    mt5.shutdown()
    time.sleep(5)

    # Start MetaTrader 5 terminal for the specific account
    subprocess.Popen([terminal_path, '/portable', '/login', str(login), '/password', password, '/server', server])
    time.sleep(10)

    for _ in range(30):  # Poll for 30 seconds, check if MT5 is running
        if mt5.initialize(login=login, password=password, server=server):
            logging.info(f"Connected to MetaTrader 5 with login: {login}")
            print(f"Connected to MetaTrader 5 with login: {login}")
            return True
        time.sleep(1)

    logging.error(f"initialize() failed, error code = {mt5.last_error()}")
    sys.exit()

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    #logging.info(f"Fetching historical data for {symbol}")
    #print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    
    # Check if data is empty or None
    if data is None or len(data) == 0:
        logging.error(f"No historical data found for {symbol} in the given date range.")
        print(f"No historical data found for {symbol}.")
        return None
    
    ohlc_data = pd.DataFrame(data)
    if 'time' not in ohlc_data.columns:
        logging.error(f"'time' column missing in the fetched data for {symbol}.")
        print(f"Error: 'time' column missing in the fetched data for {symbol}.")
        return None

    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

# Calculate mean candle size dynamically
def calculate_mean_candle_size(df):
    df['candle_size'] = df['high'] - df['low']
    mean_candle_size = df['candle_size'].mean()
    return mean_candle_size

# Add rolling features
def add_rolling_features(df, window):
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Add lag features
def add_lag_features(df, lags):
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    X_combined = pd.DataFrame(index=df['time'])

    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time", max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)

        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]

        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy/sell data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    #print(f"Fetching and processing data for {symbol}")

    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    
    if df is None or df.empty:
        logging.error(f"No historical data found for {symbol}.")
        return None, None


    # Calculate mean candle size before modifying the DataFrame
    mean_candle_size = calculate_mean_candle_size(df)

    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])

    df = df.dropna().reset_index(drop=True)

    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')

    missing_features = [feat for feat in selected_features if feat not in combined_df.columns]
    for feat in missing_features:
        combined_df[feat] = 0

    combined_df = combined_df[selected_features]

    combined_df = combined_df.dropna()

    return combined_df, mean_candle_size


# Calculate SL and TP based on entry price and specified percentages, considering Buy and Sell separately
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, trade_type):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part

    if trade_type == "Buy":
        sl_price = entry_price - risk_amount  # For Buy, SL is below the entry price
        tp_price = entry_price + reward_amount  # For Buy, TP is above the entry price
    elif trade_type == "Sell":
        sl_price = entry_price + risk_amount  # For Sell, SL is above the entry price
        tp_price = entry_price - reward_amount  # For Sell, TP is below the entry price
    else:
        raise ValueError(f"Invalid trade_type: {trade_type}. It must be 'Buy' or 'Sell'.")
    
    return sl_price, tp_price


# Place order with retry mechanism
def place_order(symbol, volume, sl_price, tp_price, trade_type, config, retry_attempts=3):
    for attempt in range(retry_attempts):
        with mt5_lock:
            tick_info = mt5.symbol_info_tick(symbol)
            if tick_info is None:
                logging.error(f"Failed to retrieve tick information for {symbol}.")
                return
            price = tick_info.ask if trade_type == "Buy" else tick_info.bid
            order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": symbol,
                "volume": volume,
                "type": order_type,
                "price": price,
                "sl": sl_price,
                "tp": tp_price,
                "deviation": 10,
                "magic": config["magic"],
                "comment": f"Python script {trade_type} order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                logging.info(f"{trade_type} order placed successfully for {symbol}.")
                print(f"{trade_type} order placed successfully for {symbol}.")
                return True
            else:
                logging.error(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")
                print(f"Order placement failed for {symbol}: {result.retcode}. Retrying...")

        time.sleep(2)  # Sleep briefly before retrying

    logging.error(f"Order failed after {retry_attempts} attempts for {symbol}")
    return False

# Predict and gather both Buy and Sell predictions first before placing any trade
def predict_and_gather_predictions(scaler, model, symbol, timeframe, start_date, end_date, features, trade_type, threshold=None):
    signals = ['WILLR_15', 'WILLR_42']  # Signals used for both buy and sell

    # Fetch and process data
    df_processed, mean_candle_size = fetch_and_process_data(symbol, timeframe, signals, features, start_date, end_date)

    if df_processed is None:
        logging.error(f"No valid data for {symbol}. Skipping predictions.")
        return None, mean_candle_size

    # Shift the data
    df_processed = df_processed.shift(periods=1, axis=0).dropna()

    # Align the features with what the scaler expects
    df_processed = df_processed[features]

    # Scale the data
    scaled_data = scaler.transform(df_processed)

    # Predict probabilities and apply the provided threshold or default to 0.5
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = (probas >= custom_threshold).astype(int)   
    pred = y_pred[-1] if len(y_pred) > 0 else 0

    X_df = pd.DataFrame(index=df_processed.index)
    X_df.index = pd.to_datetime(X_df.index)
    X_df.index = X_df.index.strftime('%Y-%m-%d %H:%M:%S')
    df_pred = pd.DataFrame(index=X_df.index)
    df_pred['prediction'] = y_pred
    df_pred.to_csv(f'pred_{symbol}_{trade_type}.csv')
    return pred, mean_candle_size  # Return last prediction and mean candle size 

# Decision function to check both Buy and Sell predictions
def decide_and_trade(buy_pred, sell_pred, symbol, volume, risk_reward_ratio, mean_candle_size, config):
    if buy_pred == 1 and sell_pred == 1:
        logging.info(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        #print(f"Both Buy and Sell predictions are 1 for {symbol}. No trade will be taken.")
        return

    if buy_pred == 1 and sell_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).ask
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Buy")
        place_order(symbol, volume, sl_price, tp_price, "Buy", config)

    if sell_pred == 1 and buy_pred == 0:
        entry_price = mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size, "Sell")
        place_order(symbol, volume, sl_price, tp_price, "Sell", config)


# Refactored function to predict both buy and sell and then decide
def process_pair(config, utc_from, utc_to, trade_decisions):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")

    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None

    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])

        with open(config["buy_features_path"], 'r') as f:
            buy_features = json.load(f)
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")

    try:
        if "sell_scaler_path" in config and "sell_model_path" in config:
            sellscaler = load(config["sell_scaler_path"])
            sellmodel = load(config["sell_model_path"])

            with open(config["sell_features_path"], 'r') as f:
                sell_features = json.load(f)
        else:
            logging.warning(f"Sell model or scaler missing for {config['symbol']}")
            print(f"Sell model or scaler missing for {config['symbol']}")
    except Exception as e:
        logging.warning(f"Error loading sell model for {config['symbol']}: {e}")
        print(f"Error loading sell model for {config['symbol']}: {e}")

    try:
        with mt5_lock:
            init_mt5_connection(config['login'], config['password'], config['server'], config['terminal_path'])

        while True:
            current_time = datetime.now(pytz.utc)

            if current_time.hour == 23 and current_time.minute == 30:  # Example trading window
                buy_pred, mean_candle_size = None, None
                sell_pred = None

                if buyscaler is not None and buymodel is not None:
                    buy_pred, mean_candle_size = predict_and_gather_predictions(
                        buyscaler, buymodel, config["symbol"], config["timeframe"], utc_from, utc_to, buy_features, "Buy",
                        threshold=config.get("buy_threshold", None)
                    )

                if sellscaler is not None and sellmodel is not None:
                    sell_pred, _ = predict_and_gather_predictions(
                        sellscaler, sellmodel, config["symbol"], config["timeframe"], utc_from, utc_to, sell_features, "Sell",
                        threshold=config.get("sell_threshold", None)
                    )

                if buy_pred is not None and sell_pred is not None:
                    decide_and_trade(buy_pred, sell_pred, config["symbol"], config["volume"], config["buy_risk_reward_ratio"], mean_candle_size, config)

                logging.info(f"Finished trading for {config['symbol']} at {current_time} UTC")

            time.sleep(60)  # Sleep for a minute and recheck
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    kill_mt5_instances()

    eurusd_config = {
        "symbol": "EURUSD",
        "login": 51988090,
        "password": '1fMdV52$74EOcw',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU1\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'EURUSD_D1_3112final/scaler.joblib',
        "buy_model_path": 'EURUSD_D1_3112final/model.joblib',
        "sell_scaler_path": 'EURUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'EURUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'EURUSD_D1_3112final/feature_names.json',
        "sell_features_path": 'EURUSD_D1_3112sell/feature_names.json',
        "magic": 1001,
    }

    gbpusd_config = {
        "symbol": "GBPUSD",
        "login": 51988092,
        "password": 'ty!H!03FNx!kEh',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU2\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112buyfinal/scaler.joblib',
        "buy_model_path": 'GBPUSD_D1_3112buyfinal/model.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112sellfinal/scaler.joblib',
        "sell_model_path": 'GBPUSD_D1_3112sellfinal/model.joblib',
        "buy_features_path": 'GBPUSD_D1_3112buyfinal/feature_names.json',
        "sell_features_path": 'GBPUSD_D1_3112sellfinal/feature_names.json',
        "magic": 1002,
    }

    usdcad_config = {
        "symbol": "USDCAD",
        "login": 51988094,
        "password": '!rwj8zZ5iA$Sz5',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU3\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:1",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "buy_features_path": 'USDCAD_D1_3112buy/feature_names.json',
        "sell_features_path": 'USDCAD_D1_3112_Sell/feature_names.json',
        "magic": 1003,
    }

    audusd_config = {
        "symbol": "AUDUSD",
        "login": 52013349,
        "password": 'kR!0XLvD61Mvsy',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU4\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'AUDUSD_D1_3112buy/scaler.joblib',
        "buy_model_path": 'AUDUSD_D1_3112buy/model.joblib',
        "sell_scaler_path": 'AUDUSD_D1_3112sell/scaler.joblib',
        "sell_model_path": 'AUDUSD_D1_3112sell/model.joblib',
        "buy_features_path": 'AUDUSD_D1_3112buy/feature_names.json',
        "sell_features_path": 'AUDUSD_D1_3112sell/feature_names.json',
        "magic": 1004,
    }

    usdchf_config = {
        "symbol": "USDCHF",
        "login": 52024094,
        "password": '8wQZWv3!vMcySk',
        "server": 'ICMarketsEU-Demo',
        "terminal_path": r"C:\Program Files\MetaTrader5ICMarketsEU5\terminal64.exe",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "buy_risk_reward_ratio": "1:2",
        "sell_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'USDCHF_D1_3112buy/scaler.joblib',
        "buy_model_path": 'USDCHF_D1_3112buy/model.joblib',
        "sell_scaler_path": 'USDCHF_D1_3112sell/scaler.joblib',
        "sell_model_path": 'USDCHF_D1_3112sell/model.joblib',
        "buy_features_path": 'USDCHF_D1_3112buy/feature_names.json',
        "sell_features_path": 'USDCHF_D1_3112sell/feature_names.json',
        "magic": 1005,
    }
    configs = [eurusd_config, gbpusd_config, usdcad_config, audusd_config,usdchf_config]

    utc_from = datetime(2023, 1, 1, tzinfo=pytz.utc)
    utc_to = datetime.now()

    trade_decisions = {config['symbol']: {'buy': 0, 'sell': 0} for config in configs}

    threads = []

    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to, trade_decisions))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()

c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from versio

Connected to MetaTrader 5 with login: 52024094
Connected to MetaTrader 5 with login: 51988094
Connected to MetaTrader 5 with login: 51988092
Connected to MetaTrader 5 with login: 51988090
Connected to MetaTrader 5 with login: 52013349


c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Feature Extraction: 100%|██████████| 38/38 [00:06<00:00,  5.64it/s]
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Feature Extraction: 100%|██████████| 38/38 [00:10<00:00,  3.65it/s]
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
R

Buy order placed successfully for USDCAD.





Rolling: 100%|██████████| 38/38 [00:08<00:00,  4.58it/s]
















Rolling: 100%|██████████| 38/38 [00:09<00:00,  3.96it/s]


Feature Extraction:   0%|          | 0/38 [00:00<?, ?it/s]


Feature Extraction:  89%|████████▉ | 34/38 [00:17<00:01,  2.63it/s]

Feature Extraction:  95%|█████████▍| 36/38 [00:18<00:00,  3.02it/s]



Feature Extraction:  97%|█████████▋| 37/38 [00:18<00:00,  2.85it/s]



Feature Extraction: 100%|██████████| 38/38 [00:19<00:00,  1.98it/s]


Feature Extraction: 100%|██████████| 38/38 [00:19<00:00,  1.93it/s]









Feature Extraction: 100%|██████████| 38/38 [00:20<00:00,  1.83it/s]




















Rolling:   0%|          | 0/38 [00:00<?, ?it/s]







































Feature Extraction: 100%|██████████| 38/38 [00:24<00:00,  1.58it/s]






Feature Extraction:   0%|          | 0/38 [00:00<?, ?it/s]


Sell order placed successfully for GBPUSD.











Rolling: 100%|██████████| 38/38 [00:07<00:00,  4.89it/s]

Feature Extraction:  76%|███████▋  | 29/38 [00:17<00:02,  3.91it/s]


Feature Extraction:  79%|███████▉  | 30/38 [00:17<00:03,  2.57it/s]

Feature Extraction:  95%|█████████▍| 36/38 [00:19<00:00,  4.59it/s]


Feature Extraction: 100%|██████████| 38/38 [00:19<00:00,  1.95it/s]

Feature Extraction: 100%|██████████| 38/38 [00:19<00:00,  1.95it/s]




Rolling:   0%|          | 0/38 [00:00<?, ?it/s]




Feature Extraction: 100%|██████████| 38/38 [00:12<00:00,  2.95it/s]


Buy order placed successfully for USDCHF.
Sell order placed successfully for AUDUSD.
